<div style="background-color:#000;"><img src="pqn.png"></img></div><div><a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.</div>

## Library installation

Install the libraries needed to fetch market data, manipulate time series, and render plots in this notebook.

In [ ]:
!pip install yfinance pandas matplotlib

We install yfinance for Yahoo Finance data and pandas for vectorized time series work. Matplotlib is included so pandas can render the comparison chart via .plot() without extra imports.

## Imports and setup

We use pandas for time-series data manipulation and plotting integration, and yfinance to download adjusted price history from Yahoo Finance.

In [ ]:
import pandas as pd
import yfinance as yf

These two libraries cover the full workflow here: fetching adjusted prices, computing returns, and preparing a clean benchmark comparison. Keeping dependencies lean makes it easier to reproduce and debug results as you iterate on your process.

## Fetch and align market data

Download adjusted close data for QQQ (benchmark) and the two holdings, then compute the benchmark’s daily returns.

In [ ]:
data = yf.download(
    ["QQQ", "AAPL", "AMZN"],
    start="2020-01-01",
    end="2024-12-31",
)

In [ ]:
closes = data.Close
benchmark_returns = closes.QQQ.pct_change()

Using adjusted prices accounts for splits and dividends so our return math is coherent. We compute daily percentage changes on QQQ to form the benchmark series we’ll subtract from the portfolio later. Clean, aligned daily returns are the foundation for any reliable active risk/return metric.

## Build portfolio and compute returns

Create an equal-share portfolio in AAPL and AMZN, compute daily PnL from position changes, and turn that into a daily return series.

In [ ]:
aapl_position = closes.AAPL * 50
amzn_position = closes.AMZN * 50

In [ ]:
portfolio_value = aapl_position + amzn_position

In [ ]:
portfolio_pnl = (
    (aapl_position - aapl_position.shift())
    + (amzn_position - amzn_position.shift())
)

In [ ]:
portfolio_returns = portfolio_pnl / portfolio_value
portfolio_returns.name = "Port"

Equal shares are simple and make it clear how price moves translate into PnL, but note this is not equal risk or equal weight. We use shift() to base PnL on changes from the prior close, avoiding lookahead. In production, verify the return denominator (many desks use prior-day value) and bake that check into tests so a small bug can’t flip your sign or scale.

## Visualize performance and compute ratio

Build cumulative growth of $1 for the portfolio and benchmark, then plot them to visually sanity-check alignment before scoring.

In [ ]:
portfolio_cumulative_returns = (portfolio_returns.fillna(0.0) + 1).cumprod()
benchmark_cumulative_returns = (benchmark_returns.fillna(0.0) + 1).cumprod()

In [ ]:
pd.concat(
    [portfolio_cumulative_returns, benchmark_cumulative_returns],
    axis=1,
).plot()

A quick cumulative plot often surfaces obvious issues like missing data, date misalignment, or unexpected drift. We start both series at 1 so visual differences reflect compounded performance. This pre-check saves time by catching data problems before we compress everything into one number.

Define and compute the information ratio from daily active returns to quantify skill per unit of active risk.

In [ ]:
def information_ratio(portfolio_returns, benchmark_returns):
    """
    Determines the information ratio of a strategy.

    Parameters
    ----------
    portfolio_returns : pd.Series or np.ndarray
        Daily returns of the strategy, noncumulative.
    benchmark_returns : int, float
        Daily returns of the benchmark or factor, noncumulative.

    Returns
    -------
    information_ratio : float

    Note
    -----
    See
    https://en.wikipedia.org/wiki/Information_ratio
    for more details.
    """
    active_return = portfolio_returns - benchmark_returns
    tracking_error = active_return.std()

    return active_return.mean() / tracking_error

In [ ]:
information_ratio(portfolio_returns, benchmark_returns)

The function subtracts benchmark returns to form active returns, then divides their mean by their volatility (tracking error). Ensure both inputs share the same dates; misalignment will dilute or distort the signal. If you want an annualized figure, multiply the daily ratio by the square root of 252, and monitor tracking error because it governs how far you can stray from the index.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.